## Set up dependencies

In [1]:
import os
import time

from dotenv import load_dotenv
import openai
import azure.cognitiveservices.speech as speechsdk


## Set up environment variables

In [2]:
load_dotenv()

True

In [3]:
openai.api_type = os.getenv('OPENAI_API_TYPE')
openai.api_key = os.getenv('OPENAI_API_KEY') 
openai.api_base = os.getenv('OPENAI_API_BASE') 
openai.api_version = os.getenv('OPENAI_API_VERSION')

In [4]:
SPEECH_API_KEY = os.getenv('SPEECH_API_KEY')
SPEECH_API_REGION = os.getenv('SPEECH_API_REGION')

COMPLETIONS_MODEL = os.getenv('COMPLETIONS_MODEL')

In [5]:
def recognize_speech_from_file(filename):
    speech_api_key = SPEECH_API_KEY
    speech_api_region = SPEECH_API_REGION

    speech_config = speechsdk.SpeechConfig(subscription=speech_api_key, region=speech_api_region)
    audio_config = speechsdk.audio.AudioConfig(filename=filename)
    # Creates a speech recognizer using a file as audio input, also specify the speech language
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config,  audio_config=audio_config)
    global done 
    done = False
    global recognized_text_list 
    recognized_text_list=[]
    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        global done
        done = True

    def recognize_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        """callback for recognizing the recognized text"""
        global recognized_text_list
        recognized_text_list.append(evt.result.text)
        print('RECOGNIZED: {}'.format(evt.result.text))

    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    speech_recognizer.recognized.connect(recognize_cb)
    speech_recognizer.session_started.connect(lambda evt: print('STT SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('STT SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    speech_recognizer.stop_continuous_recognition()

    return recognized_text_list


## Transcribe Interview to Text

In [7]:
text = recognize_speech_from_file("../data/sample.wav")
print(text)

STT SESSION STARTED: SessionEventArgs(session_id=9bcb395f3b204ba8b8e4ee5c8eb42352)
RECOGNIZING: SpeechRecognitionEventArgs(session_id=9bcb395f3b204ba8b8e4ee5c8eb42352, result=SpeechRecognitionResult(result_id=7f8c434a9b6a4212a0c11bc68d297cad, text="hello mrs ste", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=9bcb395f3b204ba8b8e4ee5c8eb42352, result=SpeechRecognitionResult(result_id=a17a583918034a47b9da81a002a17168, text="hello mrs stevens", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=9bcb395f3b204ba8b8e4ee5c8eb42352, result=SpeechRecognitionResult(result_id=5f1b4b59ffaa4a8c9a8b7c287bddbcdc, text="hello mrs stevens my name is", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=9bcb395f3b204ba8b8e4ee5c8eb42352, result=SpeechRecognitionResult(result_id=9b06ae30e6a2438a9f7637abe3bafc45, text="hello mrs stevens my name is jane", reason=ResultReason.Recognizin

## Create prompt for listing interview questions

In [8]:
prompt = f"List all the interview questions \n\n{' '.join(text)}"

openai.Completion.create(
    engine=COMPLETIONS_MODEL,
    prompt=prompt,
    temperature=0,
    max_tokens=300
)["choices"][0]["text"].strip(" \n")

"What are the responsibilities in this position? \n\nWe're looking for someone to supervise 2 office clerks, handle all the correspondence, arrange meetings and manage the front office. \n\nWhat are your salary expectations, Missus Stevens? \n\nI expect to be paid the going rate for this type of position. \n\nWhat are the office hours, Mrs. Phillips? \n\n8:30 to 4:30 with an hour off for lunch. \n\nCan you tell me about the benefits you offer? \n\nYes. We provide full medical and dental coverage, a pension plan, and a three-week holiday per year. \n\nDo you have any questions about the position? \n\nYes. \n\nWhen is the position available? \n\nWe're hoping the successful applicant can start at the beginning of next month. \n\nI see that you have excellent references. \n\nDo you have any questions about the position? \n\nYes. \n\nHave you had any supervisory experience? \n\nYes, I supervised 3 administrators in my last position. \n\nTell me about your qualifications. \n\nI can type over

In [9]:
# Record end time
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time
print(f"Execution Time: {execution_time} seconds")

Execution Time: 68.38762664794922 seconds
